<a href="https://colab.research.google.com/github/saiKelkar/Computer-Vision-Deep-Learning/blob/main/PyTorch_Quickstart.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# PyTorch has two primitives to work with data:
# torch.utils.data.DataLoader and torch.utils.data.Dataset
# Dataset -- stores the samples and their corresponding labels
# DataLoader -- wraps an iterable around the Dataset (goes through the dataset one piece at a time or in small batches)
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor

In [3]:
# Download training data from open datasets
training_data = datasets.FashionMNIST(
    root = "data",
    train = True,
    download = True,
    transform = ToTensor(),
)

# Download test data from open datasets
test_data = datasets.FashionMNIST(
    root = "data",
    train = False,
    download = True,
    transform = ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:03<00:00, 7.74MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 130kB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 2.52MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 8.64MB/s]


In [4]:
# We pass Dataset as an argument to DataLoader
# This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling, and multiprocess data loading.
# We define batch_size = 64 <-- each element in the dataloader iterable will return a batch of 64 features and labels
batch_size = 64

# Create DataLoader
train_dataloader = DataLoader(training_data, batch_size = batch_size)
test_dataloader = DataLoader(test_data, batch_size = batch_size)

for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


In [5]:
# To define a neural network, we create a class that inherits from nn.Module
# Define the layers of the network in the __init__ function,
# and specity how data will pass through the network in the forward function
# To accelerate operations in the neural network, we move it to the accelerator such as CUDA, MPS, or XPU
# Otherwise, we use CPU

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

# Define model
# nn.Module <-- base class for all neural networks in PyTorch -- provides built-in features like parameters tracking, saving / loading models, etc.
class NeuralNetwork(nn.Module):
    def __init__(self):
        # Constructor of the class -- required to properly register the model's layers
        super().__init__()
        # Flattens a 2D image (28 x 28 pixels) into a 1D vector (784 values)
        self.flatten = nn.Flatten()
        # Core of neural network inside a nn.Sequential container
        self.linear_relu_stack = nn.Sequential(
            # Input: 784 features, Output: 512 neurons
            nn.Linear(28*28, 512),
            # ReLU activation function, introducing non-linearity
            nn.ReLU(),
            # Input: 512 outputs from previous layer, Output: 512 units
            nn.Linear(512, 512),
            nn.ReLU(),
            # Final layer -- maps 512 neurons to 10 output values (representing 10 classes in the MNIST data)
            nn.Linear(512, 10)
        )

    # forward pass -- whatever happens when you give the model some input x
    def forward(self, x):
        # Image gets flattened
        x = self.flatten(x)
        # Passed through the linear_relu_stack and returns logits (raw predictions before softmax)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [7]:
# Defining loss_fn as nn.CrossEntropyLoss
loss_fn = nn.CrossEntropyLoss()
# Defining optimizer as SGD to update model weights using gradients -- Stochastic Gradient Descent
optimizer = torch.optim.SGD(model.parameters(), lr = 1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    # size -- total number of samples in the train dataset
    size = len(dataloader.dataset)
    # setting the model to train()
    model.train()
    # Loop through batches of data
    # move input (X) and labels (y) to the selected device (CPU / GPU)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        # pred is what model(X) outputs
        pred = model(X)
        # loss -- computes how far the predicted labels are from the true labels
        loss = loss_fn(pred, y)

        # Backpropagation
        # Compute gradients of the loss with respect to model parameters (backward pass)
        loss.backward()
        # Update model parameters using the gradients just calculated
        optimizer.step()
        # Clear the old gradients from the previous step before computing new ones
        optimizer.zero_grad()

        # Print loss every 100 batches
        if batch % 100 == 0:
            # Convert loss to a Python float, Number of samples processed so far
            loss, current = loss.item(), (batch + 1) * len(X)
            # loss: {loss:>7f} <-- use 7 total spaces, and align it to the right (e.g., loss: 0.021346)
            # :>5 <-- make the whole space at least 5 characters width
            # [ _ _ _ _ _ / _ _ _ _ _ ]
            print(f"loss: {loss:>7f} [{current:>5d} / {size:>5d}]")

In [9]:
# We also check the model's performance against the test dataset to ensure it is learning
def test(dataloader, model, loss_fn):
    # size -- total number of samples in the test dataset
    size = len(dataloader.dataset)
    # Number of batches in the test dataloader
    num_batches = len(dataloader)
    # set model to evaluate (disables dropouts, etc.)
    model.eval()
    # setting test_loss, and correct scores to 0
    test_loss, correct = 0, 0
    # Disable gradient calculation (for faster inference and memory efficiency)
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            # Add the loss for this batch (sum over all batches to average later)
            test_loss += loss_fn(pred, y).item()
            # Count how many predictions matched the trie labels
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    # Average the loss and converting count of correct predictions to accuracy
    test_loss /= num_batches
    correct /= size
    # (100 * correct):>0.1f <-- multiplies correct (which is a float between 0 and 1) by 100 to get a percentage accuracy
    # {test_loss:>8f} <-- right-align it in a field that's 8 characters wide (including digits and decimal point)
    print(f"Test Error: \n Accuracy: {(100 * correct):>0.1f} %, Avg loss: {test_loss:>8f}\n")

In [10]:
# The training process is conducted over several iterations (epochs)
# During each epoch, the model learns parameters to make better predictions
# We print the model's accuracy and loss at each epoch <-- we'd like to see the accuracy increase and the loss decrease with every epoch

epochs = 5
for t in range(epochs):
    print(f"Epoch {t + 1}\n - - - - - - - - - - - - - - - - -")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
 - - - - - - - - - - - - - - - - -
loss: 2.307972 [   64 / 60000]
loss: 2.291377 [ 6464 / 60000]
loss: 2.269899 [12864 / 60000]
loss: 2.264215 [19264 / 60000]
loss: 2.243737 [25664 / 60000]
loss: 2.217320 [32064 / 60000]
loss: 2.230158 [38464 / 60000]
loss: 2.188588 [44864 / 60000]
loss: 2.192180 [51264 / 60000]
loss: 2.163491 [57664 / 60000]
Test Error: 
 Accuracy: 38.3 %, Avg loss: 2.152056

Epoch 2
 - - - - - - - - - - - - - - - - -
loss: 2.164243 [   64 / 60000]
loss: 2.152910 [ 6464 / 60000]
loss: 2.092397 [12864 / 60000]
loss: 2.114351 [19264 / 60000]
loss: 2.054612 [25664 / 60000]
loss: 1.995932 [32064 / 60000]
loss: 2.034033 [38464 / 60000]
loss: 1.943626 [44864 / 60000]
loss: 1.963108 [51264 / 60000]
loss: 1.890248 [57664 / 60000]
Test Error: 
 Accuracy: 53.9 %, Avg loss: 1.880818

Epoch 3
 - - - - - - - - - - - - - - - - -
loss: 1.913208 [   64 / 60000]
loss: 1.880929 [ 6464 / 60000]
loss: 1.764738 [12864 / 60000]
loss: 1.814261 [19264 / 60000]
loss: 1.690672 [25664 /

In [11]:
# Saving model
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [12]:
# Loading models -- includes re-creating the model structure and loading the state dictionary into it
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only = True))

<All keys matched successfully>

In [18]:
# The model is now ready to make predictions
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
# Get the first test image (x) and its label (y) from test_data
x, y = test_data[5][0], test_data[5][1]
# Turn off gradient calculation
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    # Taking the index of the highest score from the output (i.e., the predicted class)
    # Getting the actual class name from the label
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    # Print the predicted and actual class names
    print(f"Predicted: '{predicted}', Actual: '{actual}'")

Predicted: 'Trouser', Actual: 'Trouser'
